In [1]:
from hedge_stand import HedgeST
from datetime import date, datetime
from demeter import TokenInfo, PoolBaseInfo, Runner, Strategy, Asset, AccountStatus, BuyAction, SellAction, RowData, \
    ChainType
import optunity
import optunity.metrics
from decimal import Decimal
import pandas as pd
import  matplotlib.pylab as plt
# import logging 
# from logging import handlers

from strategy_ploter import  plot_position_return_decomposition

%matplotlib inline

def backtest_instance(a, hedge_spread_split,hedge_spread_rate,alpha):
    # global RUNNING_TIME
    # print(f"==================spread running time {RUNNING_TIME}==================")

    decimal_a = Decimal(a).quantize(Decimal('0.00'))
    decimal_hedge_spread_split = Decimal(hedge_spread_split).quantize(Decimal('0.0'))
    decimal_hedge_spread_rate = Decimal(hedge_spread_rate).quantize(Decimal('0.00'))

    pool_id_tie500 = '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'

    pool_id_tie3000 = '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'

    eth = TokenInfo(name="eth", decimal=18)
    usdc = TokenInfo(name="usdc", decimal=6)
    pool = PoolBaseInfo(usdc, eth, 0.05, usdc)

    #收益计算基础参数
    # net_value_base = 'ETH'

    runner_instance = Runner(pool)
    # runner_instance.enable_notify = False
    runner_instance.strategy = HedgeST(decimal_a,decimal_hedge_spread_split,decimal_hedge_spread_rate,alpha)
    runner_instance.set_assets([Asset(usdc, 10000)])
    runner_instance.data_path = "../demeter/data"
    runner_instance.load_data(ChainType.Ethereum.name,
                                pool_id_tie500,
                                DATE_START,
                               DATE_END)
    runner_instance.run(enable_notify=False)

    hedge_count = runner_instance.strategy.hedge_count


    # df_status = pd.DataFrame(runner_instance.account_status_list)

    total_net_value = runner_instance.final_status.net_value
    
    final_total_usdc_value = round(total_net_value + runner_instance.strategy.e.df['total'].iloc[-1],3)
    
    final_price = runner_instance.final_status.price

    final_total_eth_value = round(final_total_usdc_value / final_price,3)

    notice = f"backtest spread:{RUNNING_TIME} times, a:{decimal_a}, hedge_spread_split:{decimal_hedge_spread_split}, hedge_spread_rate:{decimal_hedge_spread_rate},alpa:{alpha}"
    result =f"result: hedge count:{hedge_count} final_total_eth_value:{final_total_eth_value},final_total_usdc_value:{final_total_usdc_value}"
    print(notice)
    print(result)
    # if SEND_NOTICE:
    #     send_notice('CEX_Notify',notice)

    # RUNNING_TIME +=1

    # if NET_VALUE_BASE == 'USDC':
    #     print(final_total_usdc_value)
    #     return float(final_total_usdc_value)
    #     # profit_rate_usdc = profit_usdc / runner_instance.strategy.init_total_usdc
    # else:
    #     print(float(final_total_usdc_value / final_price))
    #     return float(final_total_usdc_value / final_price)


    return runner_instance





KeyboardInterrupt: 

In [ ]:

def generate_dfmerge(runner_instance):
    e = runner_instance.strategy.e

    total_net_value = runner_instance.final_status.net_value
        
    final_total_usdc_value = total_net_value + runner_instance.strategy.e.df['total'].iloc[-1]


    final_price = runner_instance.final_status.price
    # if net_value_base == 'USDC':
    profit_usdc = round(final_total_usdc_value- runner_instance.strategy.init_total_usdc,3)
    profit_rate_usdc = round(profit_usdc / runner_instance.strategy.init_total_usdc,3)
    # else:
    profit_eth = round(final_total_usdc_value/final_price - runner_instance.strategy.init_total_symbol,3)
    profit_rate_eth = round(profit_eth / runner_instance.strategy.init_total_symbol,3)

    print(f"USDC profit: {profit_usdc}, profit_rate: {profit_rate_usdc}")

    print(f"ETH profit: {profit_eth}, profit_rate: {profit_rate_eth}")

    df_status = pd.DataFrame(runner_instance.account_status_list)
    df_merge = pd.merge(e.df, df_status, on='timestamp', how='inner')

    df_merge['total_value'] = df_merge['total'] + df_merge['net_value']

    df_merge.set_index('timestamp', inplace=True)

    df_merge['total_value_profit_rate'] = df_merge['total_value'] / runner_instance.strategy.init_total_usdc


    df_merge['total_value_eth'] = df_merge['total_value'] / df_merge['price']

    df_merge['total_value_eth_profit_rate'] = df_merge['total_value_eth'] / runner_instance.strategy.init_total_symbol

    df_merge_f = df_merge.apply(pd.to_numeric, downcast='float')
   
    df_merge_f['net_value_rate'] = df_merge_f['net_value'] / df_merge_f['net_value'].iloc[0]

    df_merge_f['price_rate'] = df_merge_f['price'] / df_merge_f['price'].iloc[0]
    return df_merge_f

In [ ]:
# import matplotlib.dates as mdates

# def plot_instance(runner_instance, df_merge):

#     fig, value_ax = plt.subplots()
#     day = mdates.DayLocator(interval=7)
#     price_ax = value_ax.twinx()
#     price_ax.xaxis.set_major_locator(day)
#     price_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

#     value_ax.set_xlabel('time')
#     value_ax.set_ylabel('value', color='g')
#     price_ax.set_ylabel('price', color='b')
#     # net_value_ts = df_merge['net_value']

#     time_ts = df_merge.index
#     price_ts = [ status.price for status in runner_instance.account_status_list ]
#     total_value_eth_profit_rate_ts = df_merge['total_value_eth_profit_rate']
#     total_value_profit_rate_ts = df_merge['total_value_profit_rate']


#     value_ax.plot(time_ts,total_value_eth_profit_rate_ts,'g-',label="total_value_eth_profit_rate_ts")
#     value_ax.plot(time_ts,total_value_profit_rate_ts,'r-',label="total_value_profit_rate_ts")
#     #value_ax.plot(time_ts,value_in_account,'b-',label=" value_in broker account")
#     price_ax.plot(time_ts,price_ts,label="price")
#     fig.legend()
#     fig.show()

In [ ]:
NET_VALUE_BASE = 'ETH'
RUNNING_TIME = 1
SEND_NOTICE = False
DATE_START = date(2022, 9, 1)
DATE_END = date(2022, 10, 31)
pd.options.plotting.backend = "plotly"

instance1=backtest_instance(1.24,2.3,0.78,0.03)

df_merge = generate_dfmerge(instance1)


df_merge[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot()

In [ ]:
df_merge = generate_dfmerge(instance1)


df_merge[['price_rate','net_value_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot()

In [ ]:
instance2=backtest_instance(1.24,2.3,0.78,-1)

df_merge2 = generate_dfmerge(instance2)


df_merge2[['price_rate','total_value_eth_profit_rate','total_value_profit_rate']].plot()